In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import functools
import gym
from Config import Config
# from util import train
from Models import ActorCritic
from Networks import cnn_head_model, actor_model, critic_model, head_model
from Memory import Memory
from baselines.common.cmd_util import make_env
from baselines.common.atari_wrappers import wrap_deepmind, make_atari
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv, VecEnvWrapper
import pdb

from PPO import PPOPixel
from Networks import cnn_head_model, actor_model, critic_model, head_model
from Models import ActorCritic
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
import copy
import gym
import torch
import numpy as np
from collections import deque

import matplotlib.pyplot as plt

env_id = "BreakoutNoFrameskip-v4"
env = make_atari(env_id)
env = wrap_deepmind(env, episode_life=True, clip_rewards=True, frame_stack=True, scale=False)

# config = Config(gym.make('CartPole-v1'))
config = Config(env)

config.update_every = 500
config.num_learn = 4
config.win_condition = 230
config.n_episodes = 1000
config.max_t = 700

config.Memory = Memory
config.Model = ActorCritic
config.head_model = functools.partial(cnn_head_model, config)
config.actor_model = functools.partial(actor_model, config)
config.critic_model = functools.partial(critic_model, config)


In [3]:
class ImageToPyTorch(gym.ObservationWrapper):
    """
    Image shape to channels x weight x height
    """

    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        old_shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(
            low=0,
            high=255,
            shape=(old_shape[-1], old_shape[0], old_shape[1]),
            dtype=np.uint8,
        )

    def observation(self, observation):
        return np.transpose(observation, axes=(2, 0, 1))

def wrap_pytorch(env):
    return ImageToPyTorch(env)

In [4]:
def make_env(gym_id, seed, idx):
    def thunk():
        env = gym.make(gym_id)
       
        env = wrap_pytorch(
            wrap_deepmind(
                env,
                clip_rewards=True,
                frame_stack=True,
                scale=False,
            )
        )
        env.seed(seed)
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        return env
    return thunk

In [5]:
env = make_env(env_id, 123343534, 1)
env = env()

## PPO test

In [6]:
from PPO import PPOPixel
from Networks import cnn_head_model, actor_model, critic_model, head_model
from Models import ActorCritic
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
import copy
import gym
import torch
import numpy as np
from collections import deque

# env = copy.deepcopy(config.env)
steps = 0
scores_deque = deque(maxlen=100)
scores = []
average_scores = []
max_score = -np.Inf

agent = PPOPixel(config)

state = env.reset()
score = 0

In [7]:

# obs = torch.zeros((num_steps, 1) + env.observation_space.shape)
# actions = torch.zeros((num_steps,1 ) + env.action_space.shape)
# logprobs = torch.zeros((num_steps,1))
# rewards = torch.zeros((num_steps,1))
# dones = torch.zeros((num_steps,1))
# values = torch.zeros((num_steps,1))

# obs.shape

In [8]:
torch.FloatTensor(env.reset()).shape

torch.Size([4, 84, 84])

In [9]:
# from Models import ActorCritic

# score = 0

# num_steps = 3

# obs = torch.zeros((num_steps, 1, 4, 84, 84))
# actions = torch.zeros(num_steps, 1)
# logsprobs = torch.zeros(num_steps, 1)
# rewards = torch.zeros(num_steps)
# dones = torch.zeros(num_steps)
# values = torch.zeros(num_steps)

### Vanilla test

In [34]:
### COSTA
class Scale(nn.Module):
    def __init__(self, scale):
        super().__init__()
        self.scale = scale

    def forward(self, x):
        return x * self.scale

def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)
    return layer


class Agent(nn.Module):
    def __init__(self, envs, frames=4):
        super(Agent, self).__init__()
        self.network = nn.Sequential(
            Scale(1/255),
            layer_init(nn.Conv2d(frames, 32, 8, stride=4)),
            nn.ReLU(),
            layer_init(nn.Conv2d(32, 64, 4, stride=2)),
            nn.ReLU(),
            layer_init(nn.Conv2d(64, 64, 3, stride=1)),
            nn.ReLU(),
            nn.Flatten(),
            layer_init(nn.Linear(3136, 512)),
            nn.ReLU()
        )
        self.actor = layer_init(nn.Linear(512, envs.action_space.n), std=0.01)
        self.critic = layer_init(nn.Linear(512, 1), std=1)

    def forward(self, x):
        return self.network(x)

    def get_action(self, x, action=None):
        logits = self.actor(self.forward(x))
        probs = Categorical(logits=logits)
        if action is None:
            action = probs.sample()
        return action, probs.log_prob(action), probs.entropy()

    def get_value(self, x):
        return self.critic(self.forward(x))

In [43]:
from Models import ActorCritic

model = Agent(env)

num_steps = 10

obs = torch.zeros((num_steps, 1) + env.observation_space.shape)
print(obs.shape)
actions = torch.zeros(num_steps, 1)
logsprobs = torch.zeros(num_steps, 1)
rewards = torch.zeros(num_steps)
dones = torch.zeros(num_steps)
values = torch.zeros(num_steps)

state = env.reset()

for t in range(num_steps):
    steps += 1

    # Shape obs
    obs[t] = torch.FloatTensor(state)
    
    # Get action & save
    with torch.no_grad():
        action, log_prob, entr = model.get_action(obs[t])
    actions[t] = action
    logsprobs[t] = log_prob
    
    # Act logic & save
    next_state, reward, done, _ = env.step(action)
    rewards[t] = reward
    dones[t] = done
    
    state = next_state

    # Update 
    score += reward
logsprobs

torch.Size([10, 1, 4, 84, 84])


tensor([[-1.3838],
        [-1.3838],
        [-1.3873],
        [-1.3891],
        [-1.3891],
        [-1.3891],
        [-1.3891],
        [-1.3873],
        [-1.3838],
        [-1.3891]])

In [44]:
print("Shape obs: {}".format(obs.shape))
print("Shape action: {}".format(actions.shape))
print("Shape log probs: {}".format(logsprobs.shape))

Shape obs: torch.Size([10, 1, 4, 84, 84])
Shape action: torch.Size([10, 1])
Shape log probs: torch.Size([10, 1])


In [45]:
b_obs = obs.reshape((-1,)+env.observation_space.shape)
# b_prev_state = obs.squeeze(dim=1)
b_actions = actions.reshape((-1,)+env.action_space.shape)
b_logprobs = logsprobs.reshape(-1)

print("reshaped state space: {}".format(b_obs.shape))
print("reshaped state space: {}".format(b_actions.shape))

reshaped state space: torch.Size([10, 4, 84, 84])
reshaped state space: torch.Size([10])


In [46]:
a, lp, e = model.get_action(b_obs, b_actions)
lp.shape

torch.Size([10])

In [47]:
ratio = torch.exp(b_logprobs - lp.detach())
ratio

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [ ]:
### Daryl test

In [65]:
class ActorCritic(nn.Module):
    """Some Information about ActorCritic"""
    def __init__(self, config):
        self.config = config
        super(ActorCritic, self).__init__()

        self.head = nn.Sequential(
            nn.Conv2d(4, 32, kernel_size=8, stride=4),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, self.config.hidden_size),
            nn.ReLU(),
        )
        
        self.actor = nn.Sequential(
            nn.Linear(config.hidden_size, config.hidden_size),
            nn.ReLU()
        )
        self.actor.add_module(
            "actor_linear",
            nn.Linear(config.hidden_size, config.action_space)
        )

        self.critic = nn.Sequential(
            nn.Linear(config.hidden_size, config.hidden_size),
            nn.ReLU()
        )
        
        self.critic.add_module(
            "critic_linear",
            nn.Linear(config.hidden_size, 1)
        )

    def forward(self, x):
        x = self.head(x)
        return x

    def get_action(self, x, action=None):
        logits = self.actor(self.forward(x))
        probs = Categorical(logits=logits)
        if action is None:
            action = probs.sample()
        return action, probs.log_prob(action), probs.entropy()

In [66]:
# model = Agent(env)
model = ActorCritic(config)

num_steps = 10

obs = torch.zeros((num_steps, 1) + env.observation_space.shape)
actions = torch.zeros(num_steps, 1)
logsprobs = torch.zeros(num_steps, 1)
rewards = torch.zeros(num_steps)
dones = torch.zeros(num_steps)
values = torch.zeros(num_steps)

state = env.reset()

for t in range(num_steps):
    steps += 1

    # Shape obs
    obs[t] = torch.FloatTensor(state)
    
    # Get action & save
    with torch.no_grad():
        action, log_prob, entr = model.get_action(obs[t])
    actions[t] = action
    logsprobs[t] = log_prob
    
    # Act logic & save
    next_state, reward, done, _ = env.step(action)
    rewards[t] = reward
    dones[t] = done
    
    state = next_state

    # Update 
    score += reward
    
b_obs = obs.reshape((-1,)+env.observation_space.shape)
b_actions = actions.reshape((-1,)+env.action_space.shape)
b_logprobs = logsprobs.reshape(-1)

a, lp, e = model.get_action(b_obs, b_actions)
ratio = torch.exp(b_logprobs - lp.detach())
ratio

tensor([1.0008, 1.0018, 0.9995, 0.9989, 1.0007, 0.9994, 0.9978, 0.9983, 1.0021,
        1.0006])